<a href="https://colab.research.google.com/github/ndserra-code/Tarjamento/blob/main/nlpTarjamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 129.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import re

# Carrega o modelo de linguagem em português
#nlp = spacy.load("ptcorenews_sm")
nlp = spacy.load("pt_core_news_sm")

def funcSubstNomesProprios(texto):
    doc = nlp(texto)
    texto_modificado = texto

    # Coleta entidades do tipo ORG para preservar nomes dentro delas
    entidadesorg = [ent for ent in doc.ents if ent.label == "ORG"]

    # Coleta entidades do tipo PER (pessoa)
    nomesparaanonimizar = []
    for ent in doc.ents:
        if ent.label_ == "PER":
            # Verifica se o nome está contido em alguma entidade ORG
            dentrodeorg = any(ent.start >= org.start and ent.end <= org.end for org in entidadesorg)
            if not dentrodeorg and ent.text not in nomesparaanonimizar:
                nomesparaanonimizar.append(ent.text)

    # Substitui cada letra do nome por # - NÃO FUNCIONOU DIREITO (VERIFICAR)
    #for nome in nomesparaanonimizar:
    #   hash_nome = "#" * len(nome)
    #    textomodificado = re.sub(rf'\b{re.escape(nome)}\b', hash_nome, texto_modificado)

    # Substitui cada nome por um coringa numerado
    for i, nome in enumerate(nomesparaanonimizar, start=1):
        coringa = f"{{{{NOME_{i}}}}}"
        # Usa regex para garantir substituição exata e evitar partes de palavras quebradas (tentem melhorar pf)
        texto_modificado = re.sub(rf'\b{re.escape(nome)}\b', coringa, texto_modificado)
    return texto_modificado

In [6]:
# Versão alternativa
# Teste das hashtag (rev.02)

import spacy
import re

# Carrega o modelo de linguagem em português
#nlp = spacy.load("ptcorenews_sm")
nlp = spacy.load("pt_core_news_sm")

def funcSubstNomesProprios(texto):
    doc = nlp(texto)
    texto_modificado = texto

    # Coleta entidades do tipo ORG para preservar nomes dentro delas
    entidadesorg = [ent for ent in doc.ents if ent.label == "ORG"]

    # Coleta entidades do tipo PER (pessoa)
    nomesparaanonimizar = []
    for ent in doc.ents:
        if ent.label_ == "PER":
            # Verifica se o nome está contido em alguma entidade ORG
            dentrodeorg = any(ent.start >= org.start and ent.end <= org.end for org in entidadesorg)
            if not dentrodeorg and ent.text not in nomesparaanonimizar:
                nomesparaanonimizar.append(ent.text)

    # Substitui cada nome por um coringa numerado
    for i, nome in enumerate(nomesparaanonimizar, start=1):
        coringa = "#" * len(nome)
        #coringa = f"{{{{NOME_{i}}}}}"
        # Usa regex para garantir substituição exata e evitar partes de palavras quebradas (tentem melhorar pf)
        texto_modificado = re.sub(rf'\b{re.escape(nome)}\b', coringa, texto_modificado)
    return texto_modificado

In [8]:
# PARA TESTAR O CODIGO
varTexto = """Camila Amaral foi passear na loja MARIA AMARAL BRINQUEDOS SA. e não encontrou o que precisava. Voltou para casa e reclamou com sua amiga Joaquina do Amaral que disse que na loja da senhora Maria das Dores chamada MARIA DAS DORES BRINDES LTDA poderia encontrar o que estava procurando"""

varTextoAnonimo = funcSubstNomesProprios(varTexto)

print(varTextoAnonimo)

############# foi passear na loja MARIA AMARAL BRINQUEDOS SA. e não encontrou o que precisava. Voltou para casa e reclamou com sua amiga ################## que disse que na loja da senhora ############### chamada MARIA DAS DORES BRINDES LTDA poderia encontrar o que estava procurando
